<a href="https://colab.research.google.com/github/italo013/ETL-de-Indices/blob/main/ETL_dos_%C3%8Dndices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando Bibliotecas

In [27]:
#Manipulação dos Dados
import pandas as pd
from datetime import datetime
import requests

#Dados
import yfinance as yf

#Visualização
import plotly.graph_objects as go

!pip install openpyxl -q

## Configurações e Funções


In [28]:
data_atual = datetime.now().strftime('%d/%m/%Y')

def plot_line(df, x_col='data', y_col='rent', title="Line Plot", x_label="Date", y_label="Rent"):
    """
    Plota um gráfico de linha com os eixos x e y fornecidos.

    Parâmetros:
    df (DataFrame): DataFrame do Pandas contendo os dados
    x_col (str): Nome da coluna para o eixo x
    y_col (str): Nome da coluna para o eixo y
    title (str): Título do gráfico
    x_label (str): Nome do eixo x
    y_label (str): Nome do eixo y
    """

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df[x_col], y=df[y_col], mode='lines', name=y_col))
    fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label)
    fig.show()

def plot_candlestick(df, title="Candlestick Chart"):
    """
    Plota um gráfico de candlestick.

    Parâmetros:
    df (DataFrame): DataFrame do Pandas contendo os dados de preço.
    title (str): Título do gráfico.
    """
    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                                         open=df['Open'],
                                         high=df['High'],
                                         low=df['Low'],
                                         close=df['Close'])])

    fig.update_layout(title=title, xaxis_title='Date', yaxis_title='Price', xaxis_rangeslider_visible=False)
    fig.show()

def melt_dataframe(df, ano, column_name):
    """
    Função para reformatar e limpar o DataFrame, renomeando colunas, derretendo o DataFrame e ajustando os tipos de dados.

    Parâmetros:
    df (DataFrame): DataFrame de entrada para reformatar
    column_name (str): Nome da coluna no DataFrame derretido que vai receber o value_name

    Retorna:
    DataFrame: DataFrame reformulado e limpo
    """

    column_mapping = {f'rateValue{i}': f'{i:02}' for i in range(1, 13)}
    df = df.rename(columns=column_mapping)
    df['day'] = df['day'].astype(str)

    df_melt = pd.melt(df, id_vars=['day'], value_vars=df.columns[1:], var_name='month', value_name=column_name)

    df_melt['year'] = str(ano)
    df_melt['date'] = df_melt['year'] + '-' + df_melt['month'] + '-' + df_melt['day'].str.zfill(2)
    df_melt['date'] = pd.to_datetime(df_melt['date'], errors='coerce', format='%Y-%m-%d')

    df_final = df_melt[['date', column_name]].copy()
    if df_final[column_name].dtype == 'object':
        df_final[column_name] = df_final[column_name].str.replace('.', '').str.replace(',', '.').astype(float)
    df_final = df_final.dropna()

    return df_final

def BACEN(codigo, data_inicial, data_final, column):
  """
    Obtém dados da API do Banco Central do Brasil (BACEN). O código da série pode ser consultado aqui (https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries)

    Parameters:
    data_final (str): Data final para a qual os dados serão obtidos, no formato 'DD/MM/YYYY'.
    codigo (int, optional): Código da série de dados na API do BACEN..
    data_inicial (str, optional): Data inicial para a qual os dados que serão obtidos, no formato 'DD/MM/YYYY'.
  """
  try:
    url_bcb = f"http://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo}/dados?formato=csv&dataInicial={data_inicial}&dataFinal={data_final}"
    serie = pd.read_csv(url_bcb, sep=";")
    serie['data'] = pd.to_datetime(serie['data'], format='%d/%m/%Y')
    serie[column] = serie['valor'].str.replace(',', '.').astype(float)
    serie.drop(['valor'], axis=1, inplace=True)
    return serie

  except Exception as e:
    print(f"Erro ao acessar a API do BACEN: {e}")

def ANBIMA(codigo, column):
    """
    Obtém dados do site da ANBIMA.
    """
    url_anbima = f"https://adata-precos-prod.s3.amazonaws.com/arquivos/indices-historico/{codigo}-HISTORICO.xls"

    try:
        df = pd.read_excel(url_anbima, sheet_name='Historico')[['Data de Referência', 'Número Índice']]

        df = df.rename(columns={'Data de Referência': 'data',
                                'Número Índice': column,
                })

        df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
        return df

    except Exception as e:
        print(f"Erro ao acessar o site da ANBIMA: {e}")


## Cookies para acessar o Endpoint da B3

### IFIX

In [29]:
ifix_cookies = {
    2023:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_27_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce19882f34eba1b33a72ee3dee3d5672b252ee2dac52635fabe1eabc046d27143d8b2274cf5e8b86b6b9d4048fcdf96e435',
    '_ga': 'GA1.3.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+21%3A28%3A34+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696810467.11.1.1696811333.41.0.0',
    'TS01871345': '016e3b076fd84fd5ec79ecc347b5aef371019cfb12670689b72daedf87c785f94ca7534cb782b017574b4de1e498b3b8e454d33953',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    },
    2022:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_27_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce19882f34eba1b33a72ee3dee3d5672b252ee2dac52635fabe1eabc046d27143d8b2274cf5e8b86b6b9d4048fcdf96e435',
    '_ga': 'GA1.3.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+21%3A28%3A34+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696810467.11.1.1696811333.41.0.0',
    'TS01871345': '016e3b076fd84fd5ec79ecc347b5aef371019cfb12670689b72daedf87c785f94ca7534cb782b017574b4de1e498b3b8e454d33953',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    },
    2021:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_27_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce19882f34eba1b33a72ee3dee3d5672b252ee2dac52635fabe1eabc046d27143d8b2274cf5e8b86b6b9d4048fcdf96e435',
    '_ga': 'GA1.3.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+21%3A28%3A34+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696810467.11.1.1696811333.41.0.0',
    'TS01871345': '016e3b076fd84fd5ec79ecc347b5aef371019cfb12670689b72daedf87c785f94ca7534cb782b017574b4de1e498b3b8e454d33953',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    },
    2020:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_27_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce19882f34eba1b33a72ee3dee3d5672b252ee2dac52635fabe1eabc046d27143d8b2274cf5e8b86b6b9d4048fcdf96e435',
    '_ga': 'GA1.3.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+21%3A28%3A34+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696810467.11.1.1696811333.41.0.0',
    'TS01871345': '016e3b076f77272c591a0f18688cd9fb96d5926bce5df2c79d88dbad08959f7685b1bea9da33de9917e38bcca6ff999a7a46a6f204',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329140746.64288.0000',
    },
    2019:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_27_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce19882f34eba1b33a72ee3dee3d5672b252ee2dac52635fabe1eabc046d27143d8b2274cf5e8b86b6b9d4048fcdf96e435',
    '_ga': 'GA1.3.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+21%3A28%3A34+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696810467.11.1.1696811333.41.0.0',
    'TS01871345': '016e3b076f77272c591a0f18688cd9fb96d5926bce5df2c79d88dbad08959f7685b1bea9da33de9917e38bcca6ff999a7a46a6f204',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329140746.64288.0000',
    },
    2018:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_27_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce19882f34eba1b33a72ee3dee3d5672b252ee2dac52635fabe1eabc046d27143d8b2274cf5e8b86b6b9d4048fcdf96e435',
    '_ga': 'GA1.3.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+21%3A28%3A34+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696810467.11.1.1696811333.41.0.0',
    'TS01871345': '016e3b076f77272c591a0f18688cd9fb96d5926bce5df2c79d88dbad08959f7685b1bea9da33de9917e38bcca6ff999a7a46a6f204',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329140746.64288.0000',
    },
    2017:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_27_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce19882f34eba1b33a72ee3dee3d5672b252ee2dac52635fabe1eabc046d27143d8b2274cf5e8b86b6b9d4048fcdf96e435',
    '_ga': 'GA1.3.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+21%3A28%3A34+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696810467.11.1.1696811333.41.0.0',
    'TS01871345': '016e3b076f77272c591a0f18688cd9fb96d5926bce5df2c79d88dbad08959f7685b1bea9da33de9917e38bcca6ff999a7a46a6f204',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329140746.64288.0000',
    },
    2016:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_27_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce19882f34eba1b33a72ee3dee3d5672b252ee2dac52635fabe1eabc046d27143d8b2274cf5e8b86b6b9d4048fcdf96e435',
    '_ga': 'GA1.3.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+21%3A28%3A34+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696810467.11.1.1696811333.41.0.0',
    'TS01871345': '016e3b076f77272c591a0f18688cd9fb96d5926bce5df2c79d88dbad08959f7685b1bea9da33de9917e38bcca6ff999a7a46a6f204',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    },
    2015:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_27_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS01871345': '016e3b076f7e05ba57e19c6a827a67e94ab73a42174dd5fd0642e78b92ab9c58ec7d640289bd0660a1cf356557d5c71b70bac46bc3',
    'TS0171d45d': '011d592ce19882f34eba1b33a72ee3dee3d5672b252ee2dac52635fabe1eabc046d27143d8b2274cf5e8b86b6b9d4048fcdf96e435',
    '_ga': 'GA1.3.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+21%3A28%3A34+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696810467.11.1.1696811333.41.0.0',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329140746.64288.0000',
    },
    2014:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_27_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS01871345': '016e3b076f7e05ba57e19c6a827a67e94ab73a42174dd5fd0642e78b92ab9c58ec7d640289bd0660a1cf356557d5c71b70bac46bc3',
    'TS0171d45d': '011d592ce19882f34eba1b33a72ee3dee3d5672b252ee2dac52635fabe1eabc046d27143d8b2274cf5e8b86b6b9d4048fcdf96e435',
    '_ga': 'GA1.3.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+21%3A28%3A34+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696810467.11.1.1696811333.41.0.0',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329140746.64288.0000',
    },
    2013:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_27_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS01871345': '016e3b076f7e05ba57e19c6a827a67e94ab73a42174dd5fd0642e78b92ab9c58ec7d640289bd0660a1cf356557d5c71b70bac46bc3',
    'TS0171d45d': '011d592ce19882f34eba1b33a72ee3dee3d5672b252ee2dac52635fabe1eabc046d27143d8b2274cf5e8b86b6b9d4048fcdf96e435',
    '_ga': 'GA1.3.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+21%3A28%3A34+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696810467.11.1.1696811333.41.0.0',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329140746.64288.0000',
    },
    2012:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_27_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'TS01871345': '016e3b076f7e05ba57e19c6a827a67e94ab73a42174dd5fd0642e78b92ab9c58ec7d640289bd0660a1cf356557d5c71b70bac46bc3',
    'TS0171d45d': '011d592ce19882f34eba1b33a72ee3dee3d5672b252ee2dac52635fabe1eabc046d27143d8b2274cf5e8b86b6b9d4048fcdf96e435',
    '_ga_SS7FXRTPP3': 'GS1.1.1696810467.11.1.1696811314.60.0.0',
    '_ga': 'GA1.3.1310266363.1696351552',
    '_gat_UA-94042116-2': '1',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+21%3A28%3A34+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    },
    2011:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1ab3902b58afa0a70450c1f6444d21c7c0c13c2d191286c89d73159ee159b6ab2e7983d097ced4eb202a951de0a9481da',
    'dtCookie': 'v_4_srv_27_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+21%3A14%3A48+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696810467.11.1.1696810720.60.0.0',
    'TS01871345': '016e3b076f789a17323ea78e53286fe00b37de5178adee9f3c8127759b4f0d90db1d41fe5b744bad530e0f36fa5672b3e1e6e1f4f5',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    }
}

ifix_headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    # 'Cookie': '_gcl_au=1.1.1256438140.1696351552; TS0134a800=016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51; _ga_CNJN5WQC5G=GS1.1.1696481498.3.1.1696482095.0.0.0; _gid=GA1.3.588630146.1696653118; rxVisitor=169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0; dtSa=-; rxvt=1696700953947|1696699102537; dtPC=28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0; OptanonAlertBoxClosed=2023-10-08T01:45:30.805Z; dtCookie=v_4_srv_27_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0; TS0171d45d=011d592ce19882f34eba1b33a72ee3dee3d5672b252ee2dac52635fabe1eabc046d27143d8b2274cf5e8b86b6b9d4048fcdf96e435; _ga=GA1.3.1310266363.1696351552; OptanonConsent=isGpcEnabled=0&datestamp=Sun+Oct+08+2023+21%3A28%3A34+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B; _ga_SS7FXRTPP3=GS1.1.1696810467.11.1.1696811333.41.0.0; TS01871345=016e3b076fd84fd5ec79ecc347b5aef371019cfb12670689b72daedf87c785f94ca7534cb782b017574b4de1e498b3b8e454d33953; BIGipServerpool_sistemaswebb3-listados_8443_WAF=1329145866.64288.0000',
    'Referer': 'https://sistemaswebb3-listados.b3.com.br/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 OPR/102.0.0.0',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Opera";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

### IBrX100

In [30]:
ibrx100_cookies = {
    2023:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'TS01871345': '016e3b076f4161a257dc362c3b2688ba82337a00377ccfc528cac81e5aa39ba565dd81ca03f2a701baa5041a1895b081cb66bbab19',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329140746.64288.0000',
    },
    2022:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'TS01871345': '016e3b076f4161a257dc362c3b2688ba82337a00377ccfc528cac81e5aa39ba565dd81ca03f2a701baa5041a1895b081cb66bbab19',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329140746.64288.0000',
    },
    2021:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'TS01871345': '016e3b076f4161a257dc362c3b2688ba82337a00377ccfc528cac81e5aa39ba565dd81ca03f2a701baa5041a1895b081cb66bbab19',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329140746.64288.0000',
    },
    2020:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'TS01871345': '016e3b076f4161a257dc362c3b2688ba82337a00377ccfc528cac81e5aa39ba565dd81ca03f2a701baa5041a1895b081cb66bbab19',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329140746.64288.0000',
    },
    2019:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'TS01871345': '016e3b076f4161a257dc362c3b2688ba82337a00377ccfc528cac81e5aa39ba565dd81ca03f2a701baa5041a1895b081cb66bbab19',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329140746.64288.0000',
    },
    2018:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'TS01871345': '016e3b076f4161a257dc362c3b2688ba82337a00377ccfc528cac81e5aa39ba565dd81ca03f2a701baa5041a1895b081cb66bbab19',
    },
    2017:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'TS01871345': '016e3b076fda04b0357e2c516724ee2e086dd24ab46ae8b1abe3f5baa943afd6bed1dde04fa4010f51bb963a62d1432c094e4c5039',
    },
    2016:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'TS01871345': '016e3b076fda04b0357e2c516724ee2e086dd24ab46ae8b1abe3f5baa943afd6bed1dde04fa4010f51bb963a62d1432c094e4c5039',
    },
    2015:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'TS01871345': '016e3b076fda04b0357e2c516724ee2e086dd24ab46ae8b1abe3f5baa943afd6bed1dde04fa4010f51bb963a62d1432c094e4c5039',
    },
    2014:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'TS01871345': '016e3b076fda04b0357e2c516724ee2e086dd24ab46ae8b1abe3f5baa943afd6bed1dde04fa4010f51bb963a62d1432c094e4c5039',
    },
    2013:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'TS01871345': '016e3b076fda04b0357e2c516724ee2e086dd24ab46ae8b1abe3f5baa943afd6bed1dde04fa4010f51bb963a62d1432c094e4c5039',
    },
    2012:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'TS01871345': '016e3b076fda04b0357e2c516724ee2e086dd24ab46ae8b1abe3f5baa943afd6bed1dde04fa4010f51bb963a62d1432c094e4c5039',
    },
    2011:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'TS01871345': '016e3b076fda04b0357e2c516724ee2e086dd24ab46ae8b1abe3f5baa943afd6bed1dde04fa4010f51bb963a62d1432c094e4c5039',
    },
    2010:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'TS01871345': '016e3b076fda04b0357e2c516724ee2e086dd24ab46ae8b1abe3f5baa943afd6bed1dde04fa4010f51bb963a62d1432c094e4c5039',
    },
    2009:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'TS01871345': '016e3b076fda04b0357e2c516724ee2e086dd24ab46ae8b1abe3f5baa943afd6bed1dde04fa4010f51bb963a62d1432c094e4c5039',
    },
    2008:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'TS01871345': '016e3b076fda04b0357e2c516724ee2e086dd24ab46ae8b1abe3f5baa943afd6bed1dde04fa4010f51bb963a62d1432c094e4c5039',
    },
    2007:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'TS01871345': '016e3b076fda04b0357e2c516724ee2e086dd24ab46ae8b1abe3f5baa943afd6bed1dde04fa4010f51bb963a62d1432c094e4c5039',
    },
    2006:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'TS01871345': '016e3b076fdbd3409d97143861cba5a2ecacb28d0e9daf7679d0d0ab2961e9e725dc911951327826849b11d5dc146b637f5a7aa55d',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    },
    2005:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'TS01871345': '016e3b076fdbd3409d97143861cba5a2ecacb28d0e9daf7679d0d0ab2961e9e725dc911951327826849b11d5dc146b637f5a7aa55d',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    },
    2004:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'TS01871345': '016e3b076fdbd3409d97143861cba5a2ecacb28d0e9daf7679d0d0ab2961e9e725dc911951327826849b11d5dc146b637f5a7aa55d',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    },
    2003:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'TS01871345': '016e3b076fdbd3409d97143861cba5a2ecacb28d0e9daf7679d0d0ab2961e9e725dc911951327826849b11d5dc146b637f5a7aa55d',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    },
    2002:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'TS01871345': '016e3b076fdbd3409d97143861cba5a2ecacb28d0e9daf7679d0d0ab2961e9e725dc911951327826849b11d5dc146b637f5a7aa55d',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    },
    2001:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'TS01871345': '016e3b076fdbd3409d97143861cba5a2ecacb28d0e9daf7679d0d0ab2961e9e725dc911951327826849b11d5dc146b637f5a7aa55d',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    },
    2000:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'TS01871345': '016e3b076fdbd3409d97143861cba5a2ecacb28d0e9daf7679d0d0ab2961e9e725dc911951327826849b11d5dc146b637f5a7aa55d',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    },
    1999:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS01871345': '016e3b076f8e6d5b44a599cd5e69950657ac27ff139b62abe5163aa1f68edfd72f8e919b52963b2bbc0c3ee89ec8ea16ce1e96c6d8',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329140746.64288.0000',
    },
    1998:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'TS0171d45d': '011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696814830.22.0.0',
    '_ga': 'GA1.1.1310266363.1696351552',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    'TS01871345': '016e3b076fdbd3409d97143861cba5a2ecacb28d0e9daf7679d0d0ab2961e9e725dc911951327826849b11d5dc146b637f5a7aa55d',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    }
}

ibrx100_headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    # 'Cookie': '_gcl_au=1.1.1256438140.1696351552; TS0134a800=016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51; _ga_CNJN5WQC5G=GS1.1.1696481498.3.1.1696482095.0.0.0; _gid=GA1.3.588630146.1696653118; rxVisitor=169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0; dtSa=-; rxvt=1696700953947|1696699102537; dtPC=28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0; OptanonAlertBoxClosed=2023-10-08T01:45:30.805Z; TS0171d45d=011d592ce1b4ed0d360d93cc96c0811a6c5e13cccea09df497e4b24f743833652180231b8691a233c6a2456bab7b43b52ff3cb12e9; dtCookie=v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0; _ga_SS7FXRTPP3=GS1.1.1696814792.12.1.1696814830.22.0.0; _ga=GA1.1.1310266363.1696351552; OptanonConsent=isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A27%3A10+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B; TS01871345=016e3b076f4161a257dc362c3b2688ba82337a00377ccfc528cac81e5aa39ba565dd81ca03f2a701baa5041a1895b081cb66bbab19; BIGipServerpool_sistemaswebb3-listados_8443_WAF=1329140746.64288.0000',
    'Referer': 'https://sistemaswebb3-listados.b3.com.br/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 OPR/102.0.0.0',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Opera";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

### IDIV

In [31]:
idiv_cookies = {
    2023:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    'TS01871345': '016e3b076fedeaa3de910232fa618cc0153cff5be760287e8b0b1c1fb9c684cd93f9573e636f8e53aa44181f3303b3cde7861c092c',
    },
    2022:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    'TS01871345': '016e3b076fedeaa3de910232fa618cc0153cff5be760287e8b0b1c1fb9c684cd93f9573e636f8e53aa44181f3303b3cde7861c092c',
    },
    2021:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    'TS01871345': '016e3b076fedeaa3de910232fa618cc0153cff5be760287e8b0b1c1fb9c684cd93f9573e636f8e53aa44181f3303b3cde7861c092c',
    },
    2020:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    'TS01871345': '016e3b076fedeaa3de910232fa618cc0153cff5be760287e8b0b1c1fb9c684cd93f9573e636f8e53aa44181f3303b3cde7861c092c',
    },
    2019:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    'TS01871345': '016e3b076fedeaa3de910232fa618cc0153cff5be760287e8b0b1c1fb9c684cd93f9573e636f8e53aa44181f3303b3cde7861c092c',
    },
    2018:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    'TS01871345': '016e3b076fedeaa3de910232fa618cc0153cff5be760287e8b0b1c1fb9c684cd93f9573e636f8e53aa44181f3303b3cde7861c092c',
    },
    2017:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    'TS01871345': '016e3b076fedeaa3de910232fa618cc0153cff5be760287e8b0b1c1fb9c684cd93f9573e636f8e53aa44181f3303b3cde7861c092c',
    },
    2016:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    'TS01871345': '016e3b076fedeaa3de910232fa618cc0153cff5be760287e8b0b1c1fb9c684cd93f9573e636f8e53aa44181f3303b3cde7861c092c',
    },
    2015:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'TS01871345': '016e3b076fe051c70daa0d25c0857aabd6fa2dba5210bf48f75bd2f11ccbabcb88ce061a9033aa9a7c82675b120a22ec0efad0b798',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    },
    2014:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'TS01871345': '016e3b076fe051c70daa0d25c0857aabd6fa2dba5210bf48f75bd2f11ccbabcb88ce061a9033aa9a7c82675b120a22ec0efad0b798',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    },
    2013:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'TS01871345': '016e3b076fe051c70daa0d25c0857aabd6fa2dba5210bf48f75bd2f11ccbabcb88ce061a9033aa9a7c82675b120a22ec0efad0b798',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    },
    2012:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'TS01871345': '016e3b076fe051c70daa0d25c0857aabd6fa2dba5210bf48f75bd2f11ccbabcb88ce061a9033aa9a7c82675b120a22ec0efad0b798',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    },
    2011:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'TS01871345': '016e3b076fe051c70daa0d25c0857aabd6fa2dba5210bf48f75bd2f11ccbabcb88ce061a9033aa9a7c82675b120a22ec0efad0b798',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    },
    2010:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'TS01871345': '016e3b076fe051c70daa0d25c0857aabd6fa2dba5210bf48f75bd2f11ccbabcb88ce061a9033aa9a7c82675b120a22ec0efad0b798',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    },
    2009:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'TS01871345': '016e3b076fe051c70daa0d25c0857aabd6fa2dba5210bf48f75bd2f11ccbabcb88ce061a9033aa9a7c82675b120a22ec0efad0b798',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    },
    2008:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'TS01871345': '016e3b076fe051c70daa0d25c0857aabd6fa2dba5210bf48f75bd2f11ccbabcb88ce061a9033aa9a7c82675b120a22ec0efad0b798',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    },
    2007:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'TS01871345': '016e3b076fe051c70daa0d25c0857aabd6fa2dba5210bf48f75bd2f11ccbabcb88ce061a9033aa9a7c82675b120a22ec0efad0b798',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    },
    2006:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'TS01871345': '016e3b076fe051c70daa0d25c0857aabd6fa2dba5210bf48f75bd2f11ccbabcb88ce061a9033aa9a7c82675b120a22ec0efad0b798',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    },
    2005:{
    '_gcl_au': '1.1.1256438140.1696351552',
    'TS0134a800': '016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51',
    '_ga_CNJN5WQC5G': 'GS1.1.1696481498.3.1.1696482095.0.0.0',
    '_gid': 'GA1.3.588630146.1696653118',
    'rxVisitor': '169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0',
    'dtSa': '-',
    'rxvt': '1696700953947|1696699102537',
    'dtPC': '28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0',
    'OptanonAlertBoxClosed': '2023-10-08T01:45:30.805Z',
    'dtCookie': 'v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0',
    'TS0171d45d': '011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92',
    '_ga': 'GA1.1.1310266363.1696351552',
    'TS01871345': '016e3b076fe051c70daa0d25c0857aabd6fa2dba5210bf48f75bd2f11ccbabcb88ce061a9033aa9a7c82675b120a22ec0efad0b798',
    'BIGipServerpool_sistemaswebb3-listados_8443_WAF': '1329145866.64288.0000',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B',
    '_ga_SS7FXRTPP3': 'GS1.1.1696814792.12.1.1696816646.60.0.0',
    }
}

idiv_headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    # 'Cookie': '_gcl_au=1.1.1256438140.1696351552; TS0134a800=016e3b076f918ac04b20f87ac9de4e82692343654b625352da6c88077bf20ebcaacd514f6432512d1606db4b111a5d217c3b71ad51; _ga_CNJN5WQC5G=GS1.1.1696481498.3.1.1696482095.0.0.0; _gid=GA1.3.588630146.1696653118; rxVisitor=169669910253491OKH29S8UO3H9VGKUL9DGCG9MV2EMA0; dtSa=-; rxvt=1696700953947|1696699102537; dtPC=28$499153795_732h-vKBBESECBHFBUWNHFPWTFMQRRHEKFHHUK-0e0; OptanonAlertBoxClosed=2023-10-08T01:45:30.805Z; dtCookie=v_4_srv_24_sn_0EA94848E1CD4B7F6864A92B3D0BA7BD_perc_100000_ol_0_mul_1_app-3Afd69ce40c52bd20e_1_app-3A2fa0c7805985f6bf_1_rcs-3Acss_0; TS0171d45d=011d592ce14d053e4018ed8ff541c48bb9cd6805e6335e09aeec03d73f71a259790bc90ab5d5f1bc6088b5c8745922afffc75fef92; _ga=GA1.1.1310266363.1696351552; BIGipServerpool_sistemaswebb3-listados_8443_WAF=1329145866.64288.0000; OptanonConsent=isGpcEnabled=0&datestamp=Sun+Oct+08+2023+22%3A57%3A03+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.21.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0003%3A0%2CC0001%3A1%2CC0004%3A0%2CC0002%3A0&AwaitingReconsent=false&geolocation=%3B; _ga_SS7FXRTPP3=GS1.1.1696814792.12.1.1696816646.60.0.0; TS01871345=016e3b076fedeaa3de910232fa618cc0153cff5be760287e8b0b1c1fb9c684cd93f9573e636f8e53aa44181f3303b3cde7861c092c',
    'Referer': 'https://sistemaswebb3-listados.b3.com.br/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 OPR/102.0.0.0',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Opera";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

# Renda Fixa

## API - Banco Central (BACEN)

### SELIC

In [32]:
selic = BACEN(11, "04/06/1986", data_atual, column='selic_diaria')
selic

,data,selic_diaria
0,1986-06-04,0.065041
1,1986-06-05,0.067397
2,1986-06-06,0.066740
3,1986-06-09,0.068247
4,1986-06-10,0.067041
...,...,...
9358,2023-10-03,0.047279
9359,2023-10-04,0.047279
9360,2023-10-05,0.047279
9361,2023-10-06,0.047279


### CDI

In [33]:
cdi = BACEN(12, "06/03/1986", data_atual, column='cdi_diaria')
cdi

,data,cdi_diaria
0,1986-03-06,0.068111
1,1986-03-10,0.069028
2,1986-03-12,0.067417
3,1986-03-14,0.064584
4,1986-03-17,0.068222
...,...,...
9399,2023-10-02,0.047279
9400,2023-10-03,0.047279
9401,2023-10-04,0.047279
9402,2023-10-05,0.047279


### IPCA

In [34]:
ipca = BACEN(433, "02/01/1980", data_atual, column='ipca_mensal')
ipca

,data,ipca_mensal
0,1980-01-01,6.62
1,1980-02-01,4.62
2,1980-03-01,6.04
3,1980-04-01,5.29
4,1980-05-01,5.70
...,...,...
519,2023-04-01,0.61
520,2023-05-01,0.23
521,2023-06-01,-0.08
522,2023-07-01,0.12


## Web Scraping - Site do ANBIMA

### IMA-B

In [35]:
ima_b = ANBIMA(codigo='IMAB', column='ima_b')
ima_b['rent'] = ima_b['ima_b'].pct_change() * 100
ima_b

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,data,ima_b,rent
0,2003-09-16,723.070317,NaN
1,2003-09-17,723.162887,0.012802
2,2003-09-18,724.280592,0.154558
3,2003-09-19,726.499199,0.306319
4,2003-09-22,728.083768,0.218110
...,...,...,...
5034,2023-10-03,9347.355008,-0.635348
5035,2023-10-04,9372.219019,0.266000
5036,2023-10-05,9367.626550,-0.049001
5037,2023-10-06,9397.863260,0.322779


### IMA-B5

In [36]:
ima_b5 = ANBIMA(codigo='IMAB5', column='ima_b5')
ima_b5['rent'] = ima_b5['ima_b5'].pct_change() * 100
ima_b5

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,data,ima_b5,rent
0,2003-09-16,775.879343,NaN
1,2003-09-17,776.077621,0.025555
2,2003-09-18,777.874861,0.231580
3,2003-09-19,779.398320,0.195849
4,2003-09-22,780.464262,0.136765
...,...,...,...
5036,2023-10-03,8670.284413,-0.359680
5037,2023-10-04,8682.104402,0.136328
5038,2023-10-05,8680.775194,-0.015310
5039,2023-10-06,8694.219074,0.154870


### IMA-B5+

In [37]:
ima_b5plus = ANBIMA(codigo='IMAB5MAIS', column='ima_b5plus')
ima_b5plus['rent'] = ima_b5plus['ima_b5plus'].pct_change() * 100
ima_b5plus

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,data,ima_b5plus,rent
0,2003-09-16,715.675793,NaN
1,2003-09-17,715.758280,0.011526
2,2003-09-18,716.809345,0.146846
3,2003-09-19,719.084420,0.317389
4,2003-09-22,720.711378,0.226254
...,...,...,...
5036,2023-10-03,10715.310629,-0.893079
5037,2023-10-04,10756.874056,0.387888
5038,2023-10-05,10748.205099,-0.080590
5039,2023-10-06,10799.830286,0.480314


### IMA -G

In [38]:
ima_g = ANBIMA(codigo='IMAGERAL', column='ima_g')
ima_g['rent'] = ima_g['ima_g'].pct_change() * 100
ima_g

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,data,ima_g,rent
0,2001-12-04,532.621995,NaN
1,2001-12-05,532.986548,0.068445
2,2001-12-06,533.286808,0.056335
3,2001-12-07,533.774816,0.091509
4,2001-12-10,534.206283,0.080833
...,...,...,...
5476,2023-10-03,7622.131848,-0.253410
5477,2023-10-04,7636.221870,0.184857
5478,2023-10-05,7636.608432,0.005062
5479,2023-10-06,7648.028239,0.149540


### IRF-M

In [39]:
irf_m = ANBIMA(codigo='IRFM', column='irf_m')
irf_m['rent'] = irf_m['irf_m'].pct_change() * 100
irf_m

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,data,irf_m,rent
0,2000-02-01,863.150015,NaN
1,2000-02-02,863.786651,0.073757
2,2000-02-03,864.644959,0.099366
3,2000-02-04,865.690616,0.120935
4,2000-02-07,866.529565,0.096911
...,...,...,...
5939,2023-10-03,17091.794085,-0.291794
5940,2023-10-04,17146.290334,0.318845
5941,2023-10-05,17146.024267,-0.001552
5942,2023-10-06,17164.402961,0.107189


### IRF-M1

In [40]:
irf_m1 = ANBIMA(codigo='IRFM1', column='irf_m1')
irf_m1['rent'] = irf_m1['irf_m1'].pct_change() * 100
irf_m1

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,data,irf_m1,rent
0,2000-12-01,1000.000000,NaN
1,2000-12-04,1000.627565,0.062757
2,2000-12-05,1002.332823,0.170419
3,2000-12-06,1003.550214,0.121456
4,2000-12-07,1004.860026,0.130518
...,...,...,...
5727,2023-10-03,14786.315547,0.026434
5728,2023-10-04,14797.436854,0.075214
5729,2023-10-05,14803.867860,0.043460
5730,2023-10-06,14810.950308,0.047842


### IRF-M1+

In [41]:
irf_m1plus = ANBIMA(codigo='IRFM1MAIS', column='irf_m1plus')
irf_m1plus['rent'] = irf_m1plus['irf_m1plus'].pct_change() * 100
irf_m1plus

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,data,irf_m1plus,rent
0,2000-12-01,1000.000000,NaN
1,2000-12-04,1000.584084,0.058408
2,2000-12-05,1004.253236,0.366701
3,2000-12-06,1006.851951,0.258771
4,2000-12-07,1008.910293,0.204433
...,...,...,...
5727,2023-10-03,18820.826173,-0.437306
5728,2023-10-04,18901.899696,0.430765
5729,2023-10-05,18897.711718,-0.022156
5730,2023-10-06,18923.105301,0.134374


### IDKA IPCA 2A

In [42]:
idkaipca2a = ANBIMA(codigo='IDKAIPCA2A', column='idkaipca2a')
idkaipca2a['rent'] = idkaipca2a['idkaipca2a'].pct_change() * 100
idkaipca2a

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,data,idkaipca2a,rent
0,2005-12-30,1000.000000,NaN
1,2006-01-02,1000.650175,0.065018
2,2006-01-03,1001.905038,0.125405
3,2006-01-04,1003.095576,0.118827
4,2006-01-05,1002.528303,-0.056552
...,...,...,...
4458,2023-10-03,8142.707468,-0.327333
4459,2023-10-04,8152.837385,0.124405
4460,2023-10-05,8152.053595,-0.009614
4461,2023-10-06,8158.591358,0.080198


# Renda Variável

## API - Banco Central (BACEN)

### Câmbio (PTAX)

In [43]:
cambio = BACEN(1, "28/11/1984", data_atual, column='cambio_diaria')
cambio

,data,cambio_diaria
0,1984-11-28,2828.0000
1,1984-11-29,2828.0000
2,1984-11-30,2881.0000
3,1984-12-03,2881.0000
4,1984-12-04,2881.0000
...,...,...
9733,2023-10-03,5.1100
9734,2023-10-04,5.1526
9735,2023-10-05,5.1713
9736,2023-10-06,5.1918


## Biblioteca Yahoo Finance

### IBOV

In [44]:
ibov = yf.download('^BVSP', period="max").reset_index()
ibov

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,1993-04-27,24.799999,25.400000,24.500000,24.500000,24.500000,0
1,1993-04-28,24.500000,24.600000,23.700001,24.299999,24.299999,0
2,1993-04-29,24.299999,24.799999,23.700001,23.700001,23.700001,0
3,1993-04-30,23.700001,24.200001,23.700001,24.100000,24.100000,0
4,1993-05-03,24.100000,24.400000,23.799999,24.100000,24.100000,0
...,...,...,...,...,...,...,...
7534,2023-10-03,115055.000000,115056.000000,113151.000000,113419.000000,113419.000000,11104800
7535,2023-10-04,113430.000000,114075.000000,113036.000000,113607.000000,113607.000000,10813800
7536,2023-10-05,113609.000000,114359.000000,112705.000000,113284.000000,113284.000000,9507200
7537,2023-10-06,113283.000000,114491.000000,111599.000000,114170.000000,114170.000000,13138200


In [45]:
plot_candlestick(ibov, title="IBOVESPA Candlestick")

### IBrX50

In [46]:
ibrx50 = yf.download('^IBX50', period="max").reset_index()
ibrx50

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-12-13,8912.269531,8981.150391,8873.200195,8899.169922,8899.169922,0
1,2012-12-14,8911.450195,8999.070312,8911.450195,8990.339844,8990.339844,0
2,2012-12-17,8990.200195,9025.410156,8953.469727,8992.870117,8992.870117,0
3,2012-12-18,8993.089844,9097.919922,8992.610352,9094.910156,9094.910156,0
4,2012-12-19,9094.910156,9217.120117,9082.719727,9210.839844,9210.839844,0
...,...,...,...,...,...,...,...
2672,2023-10-03,19162.279297,19162.419922,18847.000000,18892.689453,18892.689453,0
2673,2023-10-04,18894.720703,18961.619141,18803.580078,18885.539062,18885.539062,0
2674,2023-10-05,18885.779297,18998.789062,18738.429688,18831.740234,18831.740234,0
2675,2023-10-06,18828.939453,19075.990234,18580.029297,19012.240234,19012.240234,0


In [47]:
plot_candlestick(ibrx50, title="IBrX50 Candlestick")

### Small Caps

In [48]:
smll = yf.download('SMAL11.SA', period="max").reset_index()
smll

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-04-14,28.110001,28.110001,28.110001,28.110001,28.110001,100
1,2009-04-20,27.950001,27.950001,27.670000,27.750000,27.750000,10200
2,2009-04-23,29.010000,29.010000,29.010000,29.010000,29.010000,10000
3,2009-04-24,29.570000,29.570000,29.570000,29.570000,29.570000,10000
4,2009-04-29,31.490000,31.490000,31.490000,31.490000,31.490000,100
...,...,...,...,...,...,...,...
3495,2023-10-03,99.449997,99.970001,97.019997,97.529999,97.529999,2424590
3496,2023-10-04,98.110001,99.330002,97.510002,98.290001,98.290001,2321202
3497,2023-10-05,98.410004,99.389999,96.650002,97.250000,97.250000,2559449
3498,2023-10-06,95.800003,97.139999,94.410004,96.570000,96.570000,7717473


In [49]:
plot_candlestick(smll, title="Small Cap Candlestick")

## Endpoint - B3

### IFIX

In [50]:
ano = 2023

response = requests.get('https://sistemaswebb3-listados.b3.com.br/indexStatisticsProxy/IndexCall/GetPortfolioDay/eyJpbmRleCI6IklCWFgiLCJsYW5ndWFnZSI6InB0LWJyIiwieWVhciI6IjIwMjMifQ==',
    cookies=ifix_cookies[ano],
    headers=ifix_headers,
)

ifix = response.json().get('results')
ifix = pd.json_normalize(ifix)
ifix = melt_dataframe(ifix, ano, 'ifix')
ifix

<ipython-input-28-156fdfdf0abd>:62: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



,date,ifix
1,2023-01-02,45204.03
2,2023-01-03,44293.15
3,2023-01-04,44768.09
4,2023-01-05,45715.83
5,2023-01-06,46285.43
...,...,...
281,2023-10-03,47772.26
282,2023-10-04,47829.58
283,2023-10-05,47686.14
284,2023-10-06,48054.41


### IBrX100

In [58]:
ano = 2023

response = requests.get('https://sistemaswebb3-listados.b3.com.br/indexStatisticsProxy/IndexCall/GetPortfolioDay/eyJpbmRleCI6IklCWFgiLCJsYW5ndWFnZSI6InB0LWJyIiwieWVhciI6IjE5OTkifQ==',
    cookies=ibrx100_cookies[ano],
    headers=ibrx100_headers,
)

ibrx100 = response.json().get('results')
ibrx100 = pd.json_normalize(ibrx100)
ibrx100 = melt_dataframe(ibrx100, ano, 'ibrx100')
ibrx100

<ipython-input-28-156fdfdf0abd>:62: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



,date,ibrx100
3,2023-01-04,1293.26
4,2023-01-05,1309.50
5,2023-01-06,1341.76
6,2023-01-07,1286.00
7,2023-01-08,1263.17
...,...,...
363,2023-12-23,3035.82
367,2023-12-27,3037.85
368,2023-12-28,3095.29
369,2023-12-29,3164.47


### IDIV

In [57]:
ano = 2011

response = requests.get('https://sistemaswebb3-listados.b3.com.br/indexStatisticsProxy/IndexCall/GetPortfolioDay/eyJpbmRleCI6IklESVYiLCJsYW5ndWFnZSI6InB0LWJyIiwieWVhciI6IjIwMjMifQ==',
    cookies=idiv_cookies[ano],
    headers=idiv_headers,
)

idiv = response.json().get('results')
idiv = pd.json_normalize(idiv)
idiv = melt_dataframe(idiv, ano, 'idiv')
idiv

<ipython-input-28-156fdfdf0abd>:62: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



,date,idiv
1,2011-01-02,6982.97
2,2011-01-03,6864.69
3,2011-01-04,6947.87
4,2011-01-05,7072.03
5,2011-01-06,7133.53
...,...,...
281,2011-10-03,7774.67
282,2011-10-04,7754.56
283,2011-10-05,7721.48
284,2011-10-06,7759.77
